In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import date, timedelta,datetime


plt.style.use('default')
%matplotlib inline


In [ ]:
#Running Date
run_date = str(date.today())
run_date_ts=datetime.now()

run_date_name = run_date.split("-")[0] + run_date.split("-")[1] + run_date.split("-")[2]
# print(run_date_name)
# print(run_date)
print(run_date_ts)

2024-06-12 18:26:36.379072


In [ ]:
df=pd.read_csv('forecasting_data_8_may.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183774 entries, 0 to 183773
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   CALDAY      183774 non-null  object 
 1   PRODUCT_ID  183774 non-null  int64  
 2   STORE_ID    183774 non-null  object 
 3   SALE_QTY    183774 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.6+ MB


In [ ]:
print(df['PRODUCT_ID'].nunique())
print(df['STORE_ID'].nunique())


156
111


In [ ]:
df

,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY
0,2024-04-28,10180850,2052,1.0
1,2024-04-28,10180850,2036,0.0
2,2024-04-28,10007930,2036,0.0
3,2024-04-28,10142036,2036,1.0
4,2024-04-28,10008690,2052,1.0
...,...,...,...,...
183769,2024-03-15,10141429,5338,1.0
183770,2024-03-15,10142282,6591,1.0
183771,2024-03-15,10142283,5155,1.0
183772,2024-03-15,10013259,6694,1.0


In [ ]:
# Basic Data Processing
df['SALE_QTY'] = df['SALE_QTY'].fillna(0)

# Negative values
df.loc[df['SALE_QTY']<0,'SALE_QTY' ] = 0
df[df['SALE_QTY']<0]
df.shape

(183774, 4)

In [ ]:
df['PRODUCT_ID']=df['PRODUCT_ID'].astype(str)

In [ ]:
# PreProcessing Spark
df.drop_duplicates(inplace = True)
df['product_store']=df.PRODUCT_ID + "_" + df.STORE_ID
# df.shape
df.head(5)

,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY,product_store
0,2024-04-28,10180850,2052,1.0,10180850_2052
1,2024-04-28,10180850,2036,0.0,10180850_2036
2,2024-04-28,10007930,2036,0.0,10007930_2036
3,2024-04-28,10142036,2036,1.0,10142036_2036
4,2024-04-28,10008690,2052,1.0,10008690_2052


In [ ]:
Predict_Value=df['product_store'].unique()
print(len(Predict_Value),Predict_Value)

13446 ['10180850_2052' '10180850_2036' '10007930_2036' ... '10013007_6009'
 '10142076_4229' '10197816_3294']


In [ ]:
x=df.groupby(['product_store'])['SALE_QTY'].sum()
print(x[x==0].count())
#For all values of x we predict zero irrespective of any model because their total sales is zero in entire period.

830


In [ ]:
z=x[x==0].index.unique()
print(z)


Index(['10007926_3321', '10007926_3327', '10007926_3445', '10007926_3922',
       '10007926_4383', '10007926_4808', '10007926_5793', '10007926_6694',
       '10007930_5983', '10008681_6694',
       ...
       '10617348_5854', '10617348_6030', '10617348_6259', '10617348_6267',
       '10617348_6315', '10617348_6408', '10617348_6409', '10617348_6615',
       '10624152_6408', '10638471_6770'],
      dtype='object', name='product_store', length=830)


In [ ]:
y=x[x!=0].index.unique()
print(len(y))
print(y)

12616
Index(['10007925_2036', '10007925_2052', '10007925_2061', '10007925_2107',
       '10007925_2531', '10007925_2682', '10007925_2817', '10007925_2A00',
       '10007925_2A49', '10007925_3016',
       ...
       '10639853_6518', '10639853_6591', '10639853_6615', '10639853_6673',
       '10639853_6694', '10639853_6770', '10639853_6826', '10639853_6829',
       '10639853_6831', '10639853_6921'],
      dtype='object', name='product_store', length=12616)


In [ ]:
# # Basic Feature engineering
# def date_features(df):
#     # Date Features
#     df['date'] = pd.to_datetime(df['CALDAY'])
#     df['year'] = df.date.dt.year
#     df['month'] = df.date.dt.month
#     df['day'] = df.date.dt.day
#     df['dayofyear'] = df.date.dt.dayofyear
#     df['dayofweek'] = df.date.dt.dayofweek
#     #df['weekofyear'] = df.date.dt.weekofyear

#     # Additionnal Data Features
#     df['day^year'] = np.log((np.log(df['dayofyear'] + 1)) ** (df['year'] - 2000))

#     # Drop date
#     df.drop('date', axis=1, inplace=True)

#     return df

# # Dates Features for Train, Test
# df=date_features(df)
df['CALDAY']=pd.to_datetime(df['CALDAY']).dt.date
df=df.sort_values(by=['CALDAY'])

df.tail(5)

,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY,product_store
38915,2024-05-08,10141686,6259,1.0,10141686_6259
38914,2024-05-08,10323684,5983,1.0,10323684_5983
38913,2024-05-08,10009164,5591,1.0,10009164_5591
102926,2024-05-08,10180851,3979,3.0,10180851_3979
41554,2024-05-08,10007927,4250,1.0,10007927_4250


In [ ]:
def mape2(df,col,n):

    y_true, y_pred=df.loc[:,'SALE_QTY'].tail(n),df.loc[:,col].tail(n)
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    if y_true.sum() != 0:
      return (np.abs((y_true - y_pred)).sum() / y_true.sum()) * 100
    else:
      return 121
# mape2(test_data_forecast,col='SMA10',n=9)

In [ ]:
# print(mape2(test_data_forecast,col='CMA10',n=9))

In [ ]:

def add_rolling(df, window, min_periods, indexes, value, col):

    rolling_avg = df.groupby(indexes)[value].rolling(window=window, min_periods=min_periods).mean().reset_index(level=indexes, drop=True)  # Calculate rolling mean
    df[col] = rolling_avg  # Add the rolling mean values as a new column
    df[col]=df[col].shift(1)
    return df

In [ ]:
# def cma (df,w,col):
#   weights = np.arange(w, 0, -1)
#   for i in range(w,len(df)):
#     x=0
#     for j in range(len(weights)):
#       x +=  weights[j]*float(df.loc[i-j,'SALE_QTY'])
#     x=x/np.sum(weights)
#     df.loc[i,col]=x
#   return df
#   # df[col]=df[col].shift(w)



In [ ]:
# def cma(df, w, col):
#     # Create an array of weights in descending order
#   weights = np.arange(w, 0, -1)

#     # Initialize the new column with NaNs
#   df[col] = np.nan

#     # Iterate over the DataFrame to calculate the CMA
#   for i in range(w-1, len(df)):
#     weighted_sum = 0
#     for j in range(w):
#       weighted_sum += weights[j] * df.loc[i-j,'SALE_QTY']
#     weighted_average = weighted_sum / np.sum(weights)
#     df.at[i, col] = weighted_average

#   return df

In [ ]:

def cma(df, w, col):
  # Create an array of weights in descending order
  weights = np.arange(w, 0, -1)

  # Initialize the new column with NaNs
  df[col] = np.nan

  # Iterate over the DataFrame to calculate the CMA
  for i in range(w-1, len(df)):
    weighted_sum = 0
    for j in range(w):
      c=i-j
      weighted_sum += weights[j] * df.at[c,'SALE_QTY']
    weighted_average = weighted_sum / np.sum(weights)
    df.loc[i, col] = weighted_average
  return df



In [ ]:

def predict_weighted_average(df, w, col, value_col='SALE_QTY'):
    # Create an array of weights in descending order
    weights = np.arange(w, 0, -1)

    # Initialize the new column with NaNs
    df[col] = np.nan

    # Iterate over the DataFrame to calculate the predicted weighted average
    for i in range(w,len(df)):
        weighted_sum = 0
        for j in range(w):
            weighted_sum += weights[j] * df.iloc[i-j][value_col]
        weighted_average = weighted_sum / np.sum(weights)
        df.at[i, col] = weighted_average

    return df

In [ ]:
def ewa(df, alpha, col):
    # Initialize the first element of the EWA column
    df[col] = 0.0
    df.iloc[0, df.columns.get_loc(col)] = df.iloc[0, df.columns.get_loc('SALE_QTY')]

    # Calculate the EWA for the rest of the elements
    for i in range(1, len(df)):
        df.iloc[i, df.columns.get_loc(col)] = (
            (1 - alpha) * df.iloc[i - 1, df.columns.get_loc(col)] +
            alpha * df.iloc[i, df.columns.get_loc('SALE_QTY')]
        )

    return df


In [ ]:
Test_result=pd.DataFrame()

In [ ]:
start_date = df['CALDAY'].min()
end_date = df['CALDAY'].max()
print(start_date,end_date)


2024-03-10 2024-05-08


In [ ]:
for s in range(0,1):
  g=y[s]
  data=df[df['product_store']==g].copy()

  data.set_index('CALDAY', inplace=True)
  full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
  data = data.reindex(full_date_range)

  # Fill missing values
  data['SALE_QTY'].fillna(0, inplace=True)
  data['product_store']=g
  # data.sort_index(inplace=True)

  data['SMA7'] = data['SALE_QTY'].rolling(window=7).mean()
  data['SMA14'] = data['SALE_QTY'].rolling(window=14).mean()
  data['SMA21'] = data['SALE_QTY'].rolling(window=21).mean()
  data['SMA30'] = data['SALE_QTY'].rolling(window=30).mean()


  # data=add_rolling(data,7,1,['STORE_ID', 'PRODUCT_ID'],'SALE_QTY','SMA7')
  # data=add_rolling(data,14,1,['STORE_ID', 'PRODUCT_ID'],'SALE_QTY','SMA14')
  # data=add_rolling(data,21,1,['STORE_ID', 'PRODUCT_ID'],'SALE_QTY','SMA21')
  # data=add_rolling(data,30,1,['STORE_ID', 'PRODUCT_ID'],'SALE_QTY','SMA30')
  # data=cma(data,7,'CMA7')
  # data=cma(data,w=14,col='CMA14')
  # data=cma(data,w=21,col='CMA21')

  data = predict_weighted_average(data, 5, 'CMA7')

  data=ewa(data,.25,'EWA7')
  data=ewa(data,.125,'EWA14')
  data=ewa(data,.08,'EWA21')
  # data=exponetial_weighted_ma(data,7,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA21')
  # data=exponetial_weighted_ma(data,14,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA14')
  # data=exponetial_weighted_ma(data,21,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA21')

  # Testing values
  # Test_result.loc[s, 'product_store'] = g
  # Test_result.loc[s,'SMA7']=mape2(data, 'SMA7',n=9)
  # Test_result.loc[s,'SMA14']=mape2(data, 'SMA14',n=9)
  # Test_result.loc[s,'SMA21']=mape2(data, 'SMA21',n=9)
  # Test_result.loc[s,'SMA30']=mape2(data, 'SMA30',n=9)
  # Test_result.loc[s,'CMA7']= mape2(data, 'CMA7',n=9)
  # Test_result.loc[s,'CMA14']=mape2(data, 'CMA14',n=9)
  # Test_result.loc[s,'CMA21']=mape2(data, 'CMA21',n=9)
  # Test_result.loc[s,'EWA7']=mape2(data, 'EWA7',n=9)
  # Test_result.loc[s,'EWA14']=mape2(data, 'EWA14',n=9)
  # Test_result.loc[s,'EWA21']=mape2(data, 'EWA21',n=9)

  # # Checking minimum value
  # min_row(Test_result,n=s)

  # #Prediction on lowest model
  # pred(Test_result,data,s)


In [ ]:
# data.drop(columns=['EWA1','EWA2'], inplace=True)
data.head(15)

,PRODUCT_ID,STORE_ID,SALE_QTY,product_store,SMA7,SMA14,SMA21,SMA30,CMA7,EWA7,EWA14,EWA21
2024-03-10 00:00:00,10007925,2036,1.0,10007925_2036,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000
2024-03-11 00:00:00,10007925,2036,5.0,10007925_2036,NaN,NaN,NaN,NaN,NaN,2.000000,1.500000,1.320000
2024-03-12 00:00:00,10007925,2036,2.0,10007925_2036,NaN,NaN,NaN,NaN,NaN,2.000000,1.562500,1.374400
2024-03-13 00:00:00,10007925,2036,0.0,10007925_2036,NaN,NaN,NaN,NaN,NaN,1.500000,1.367188,1.264448
2024-03-14 00:00:00,10007925,2036,0.0,10007925_2036,NaN,NaN,NaN,NaN,NaN,1.125000,1.196289,1.163292
2024-03-15 00:00:00,10007925,2036,0.0,10007925_2036,NaN,NaN,NaN,NaN,NaN,0.843750,1.046753,1.070229
2024-03-16 00:00:00,10007925,2036,0.0,10007925_2036,1.142857,NaN,NaN,NaN,NaN,0.632812,0.915909,0.984610
2024-03-17 00:00:00,10007925,2036,0.0,10007925_2036,1.000000,NaN,NaN,NaN,NaN,0.474609,0.801420,0.905842
2024-03-18 00:00:00,10007925,2036,0.0,10007925_2036,0.285714,NaN,NaN,NaN,NaN,0.355957,0.701243,0.833374
2024-03-19 00:00:00,10007925,2036,0.0,10007925_2036,0.000000,NaN,NaN,NaN,NaN,0.266968,0.613587,0.766704


In [ ]:
data.to_csv('x')

In [ ]:
Test_result.head(10)

,product_store,EWA21,SMA7,SMA14,SMA21,SMA30,CMA7,CMA14,CMA21,EWA7,EWA14
0,10007925_2036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10007925_2052,121.0,121.0,121.0,121.0,121.0,121.0,121.0,121.0,121.0,121.0


In [ ]:
Test_result.to_csv("Test_result.csv",index=False)

NameError: name 'Test_result' is not defined

In [ ]:
df.head()


,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY,product_store,year,month,day,dayofyear,dayofweek,day^year
155838,2024-03-10,10639853,3388,2.0,10639853_3388,2024,3,10,70,6,34.797553
153423,2024-03-10,10008738,6831,1.0,10008738_6831,2024,3,10,70,6,34.797553
153422,2024-03-10,10142036,6826,3.0,10142036_6826,2024,3,10,70,6,34.797553
153421,2024-03-10,10197693,6831,0.0,10197693_6831,2024,3,10,70,6,34.797553
38412,2024-03-10,10180851,6829,0.0,10180851_6829,2024,3,10,70,6,34.797553


In [ ]:

df_forecast=df.copy()
df_forecast=df_forecast[df_forecast['product_store']=='10007925_2036']
df_forecast = add_rolling(df_forecast, 7, 1, ['STORE_ID', 'PRODUCT_ID'], 'SALE_QTY', 'SMA7', 'dayofyear')

cma(df_forecast,w=7,col='CMA7')
cma(df_forecast,w=14,col='CMA14')
cma(df_forecast,w=21,col='CMA21')
# df_forecast = sma(df=df_forecast, days=5, value_col='SALE_QTY', sma_col='SMA6')

df_forecast=add_rolling(df_forecast,14,1,['STORE_ID', 'PRODUCT_ID'],'SALE_QTY','SMA14','dayofyear')
df_forecast=add_rolling(df_forecast,21,1,['STORE_ID', 'PRODUCT_ID'],'SALE_QTY','SMA21','dayofyear')
df_forecast=add_rolling(df_forecast,30,1,['STORE_ID', 'PRODUCT_ID'],'SALE_QTY','SMA30','dayofyear')

# df_forecast = add_cummulative5(df_forecast, 5, 1, ['STORE_ID', 'PRODUCT_ID'], 'SALE_QTY', 'CMA5', 'dayofyear')
# df_forecast['CMA5'] = df_forecast['SALE_QTY'].rolling(window=5, min_periods=1).mean()
# df_forecast['CMA10'] = df_forecast['SALE_QTY'].rolling(window=10, min_periods=1).mean()
# df_forecast['CMA15'] = df_forecast['SALE_QTY'].rolling(window=15, min_periods=1).mean()
# df_forecast=add_cummulative5(df_forecast,5,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','CMA5','dayofyear')
# df_forecast=add_cummulative(df_forecast,10,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','CMA10','dayofyear')
# df_forecast=add_cummulative15(df_forecast,15,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','CMA15','dayofyear')
df_forecast=exponetial_weighted_ma(df_forecast,7,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA7','dayofyear')
df_forecast=exponetial_weighted_ma(df_forecast,14,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA14','dayofyear')
df_forecast=exponetial_weighted_ma(df_forecast,21,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA21','dayofyear')
# Remove NULL values
# df_max.dropna(inplace=True)
test_data_forecast=df_forecast.tail(9)

df_forecast.head(20)
# test_data_forecast.head(5)

,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY,product_store,year,month,day,dayofyear,dayofweek,...,SMA7,CMA7,CMA14,CMA21,SMA14,SMA21,SMA30,EWMA7,EWMA14,EWMA21
2596,2024-03-10,10007925,2036,1.0,10007925_2036,2024,3,10,70,6,...,NaN,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
92003,2024-03-11,10007925,2036,5.0,10007925_2036,2024,3,11,71,0,...,1.000000,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
101573,2024-03-12,10007925,2036,2.0,10007925_2036,2024,3,12,72,1,...,3.000000,0.000000,0.000000,0.0,3.000000,3.000000,3.000000,3.133333,3.068966,3.046512
122441,2024-03-13,10007925,2036,0.0,10007925_2036,2024,3,13,73,2,...,2.666667,0.000000,0.000000,0.0,2.666667,2.666667,2.666667,2.704142,2.687797,2.681327
102502,2024-03-14,10007925,2036,0.0,10007925_2036,2024,3,14,74,3,...,2.000000,0.000000,0.000000,0.0,2.000000,2.000000,2.000000,1.887316,1.944795,1.963520
154183,2024-03-15,10007925,2036,0.0,10007925_2036,2024,3,15,75,4,...,1.600000,0.000000,0.000000,0.0,1.600000,1.600000,1.600000,1.402982,1.500404,1.533456
17112,2024-03-16,10007925,2036,0.0,10007925_2036,2024,3,16,76,5,...,1.333333,0.000000,0.000000,0.0,1.333333,1.333333,1.333333,1.084820,1.205314,1.247263
78524,2024-03-17,10007925,2036,0.0,10007925_2036,2024,3,17,77,6,...,1.142857,0.321429,0.000000,0.0,1.142857,1.142857,1.142857,0.861534,0.995533,1.043281
137881,2024-03-18,10007925,2036,0.0,10007925_2036,2024,3,18,78,0,...,1.000000,0.071429,0.000000,0.0,1.000000,1.000000,1.000000,0.697467,0.839066,0.890680
125258,2024-03-19,10007925,2036,0.0,10007925_2036,2024,3,19,79,1,...,0.285714,0.000000,0.000000,0.0,0.888889,0.888889,0.888889,0.572802,0.718135,0.772331


In [ ]:
Test_result=pd.DataFrame()
# Test_result.loc['product_store']='10007925_2036'
a=0
Test_result.loc[a, 'product_store'] = '10007925_2036'
Test_result.loc[a,'SMA7']=mape2(test_data_forecast, 'SMA7',n=9)
Test_result.loc[a,'SMA14']=mape2(test_data_forecast, 'SMA14',n=9)
Test_result.loc[a,'SMA21']=mape2(test_data_forecast, 'SMA21',n=9)
Test_result.loc[a,'SMA30']=mape2(test_data_forecast, 'SMA30',n=9)
Test_result.loc[a,'CMA7']= mape2(test_data_forecast, 'CMA7',n=9)
Test_result.loc[a,'CMA14']=mape2(test_data_forecast, 'CMA14',n=9)
Test_result.loc[a,'CMA21']=mape2(test_data_forecast, 'CMA21',n=9)
Test_result.loc[a,'EWMA7']=mape2(test_data_forecast, 'EWMA7',n=9)
Test_result.loc[a,'EWMA14']=mape2(test_data_forecast, 'EWMA14',n=9)
Test_result.loc[a,'EWMA21']=mape2(test_data_forecast, 'EWMA21',n=9)
Test_result.head()


,product_store,SMA7,SMA14,SMA21,SMA30,CMA7,CMA14,CMA21,EWMA7,EWMA14,EWMA21
0,10007925_2036,107.086168,98.29932,112.912572,116.618166,81.67517,88.265306,92.296548,103.305816,101.360275,98.617331


In [ ]:
def min_row(df,n):

  a=np.array(df.iloc[n,1:10])
  min_index=np.argmin(a)
  df.loc[n,'min_index']= min_index
min_row(Test_result,n=0)
Test_result.head()

,product_store,EWA21,SMA7,SMA14,SMA21,SMA30,CMA7,CMA14,CMA21,EWA7,EWA14,min_index
0,10007925_2036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,10007925_2052,121.0,121.0,121.0,121.0,121.0,121.0,121.0,121.0,121.0,121.0,NaN


In [ ]:
# ENCODINH 0=SMA10
# 2=SMA5
# SIMILARLY 4 =CMA5

def pred(df1,df2,n):
  #For SMA 7
  if df1.loc[n,'min_index']==0:
    w=7
    a=np.array(df2['SALE_QTY'].tail(w))
    for i in range(1,11):
      b='Prediction'+ str(i)
      x= np.mean(a[-7:])
      a=np.append(a,x)
      df1.loc[n,b]=x

   #For SMA 14
  elif df1.loc[n,'min_index']==1:
    w=14
    a=np.array(df2['SALE_QTY'].tail(w))
    for i in range(1,11):
      b='Prediction'+ str(i)
      x= np.mean(a[-14:])
      a=np.append(a,x)
      df1.loc[n,b]=x

  #For SMA 15
  elif df1.loc[n,'min_index']==2:
    w=21
    a=np.array(df2['SALE_QTY'].tail(w))
    for i in range(1,11):
      b='Prediction'+ str(i)
      x= np.mean(a[-21:])
      a=np.append(a,x)
      df1.loc[n,b]=x

 # for SMA 30
  elif df1.loc[n,'min_index']==3:
    w=30
    a=np.array(df2['SALE_QTY'].tail(w))
    for i in range(1,11):
      b='Prediction'+ str(i)
      x= np.mean(a[-30:])
      a=np.append(a,x)
      df1.loc[n,b]=x

#  FOR CMA7
  elif df1.loc[n,'min_index']==4:
    # implieS CMA5
    w=7
    a=np.array(df2['SALE_QTY'].tail(w))
    weights = np.arange(w, 0, -1)

    for i in range(w,w+10):
      b='Prediction'+ str(i-6)
      x=0
      for j in range(len(weights)):
        x +=  weights[j]*a[i-j-1]
      x=x/np.sum(weights)
      a = np.append(a,x)
      df1.loc[n,b]=x

# FOR CMA10
  elif df1.loc[n,'min_index']==5:
    w=14
    a=np.array(df2['SALE_QTY'].tail(w))
    weights = np.arange(w, 0, -1)

    for i in range(w,w+10):
      b='Prediction'+ str(i-13)
      x=0
      for j in range(len(weights)):
        x +=  weights[j]*a[i-j-1]
      x=x/np.sum(weights)
      a = np.append(a,x)
      df1.loc[n,b]=x

#FOR CMA15
  elif df1.loc[n,'min_index']==6:
    w=21
    a=np.array(df2['SALE_QTY'].tail(w))
    weights = np.arange(w, 0, -1)

    for i in range(w,w+10):
      b='Prediction'+ str(i-20)
      x=0
      for j in range(len(weights)):
        x +=  weights[j]*a[i-j-1]
      x=x/np.sum(weights)
      a = np.append(a,x)
      df1.loc[n,b]=x

  #FOR ema15
  elif df1.loc[n,'min_index']==7:
    for i in range(1,11):
      b='Prediction'+ str(i)
      df1.loc[n,b]=



SyntaxError: incomplete input (<ipython-input-40-45d5f205eec8>, line 94)

In [ ]:
Test_result.head()

NameError: name 'Test_result' is not defined

In [ ]:
x=MAPE_EVAL(test_data_forecast, 'CMA5')
print(x)

42.86


In [ ]:
df_forecast.tail(10)

,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY,product_store,year,month,day,dayofyear,dayofweek,...,SMA10,SMA5,SMA15,SMA30,CMA5,CMA10,CMA15,EWMA5,EWMA10,EWMA30
78095,2024-04-24,10007925,2036,4.4469,10007925_2036,2024,4,24,115,2,...,3.0,3.6,3.266667,3.600000,3.6,3.0,3.266667,3.591989,3.623552,3.328183
15319,2024-04-25,10007925,2036,4.5084,10007925_2036,2024,4,25,116,3,...,3.0,1.8,3.133333,3.666667,1.8,3.0,3.133333,2.992917,3.286695,3.181236
160991,2024-04-27,10007925,2036,4.5172,10007925_2036,2024,4,27,118,5,...,2.7,1.8,2.866667,3.666667,1.8,2.7,2.866667,2.827337,3.167325,3.129695
91746,2024-04-28,10007925,2036,4.4798,10007925_2036,2024,4,28,119,6,...,3.6,3.8,3.600000,4.066667,3.8,3.6,3.600000,2.522637,2.966631,3.037814
49926,2024-05-01,10007925,2036,4.4062,10007925_2036,2024,5,1,122,2,...,4.1,4.4,3.733333,4.300000,4.4,4.1,3.733333,4.102819,3.801709,3.420302
139870,2024-05-03,10007925,2036,4.3088,10007925_2036,2024,5,3,124,4,...,4.4,5.2,3.733333,4.433333,5.2,4.4,3.733333,4.585841,4.096918,3.571501
68789,2024-05-04,10007925,2036,4.1962,10007925_2036,2024,5,4,125,5,...,3.5,5.2,3.733333,4.333333,5.2,3.5,3.733333,4.488174,4.087984,3.589421
174320,2024-05-05,10007925,2036,4.0754,10007925_2036,2024,5,5,126,6,...,3.6,5.4,3.600000,4.033333,5.4,3.6,3.600000,4.073384,3.895770,3.523581
77235,2024-05-06,10007925,2036,3.9529,10007925_2036,2024,5,6,127,0,...,3.5,3.2,3.466667,4.033333,3.2,3.5,3.466667,3.727754,3.721451,3.461040
125711,2024-05-08,10007925,2036,3.8361,10007925_2036,2024,5,8,129,2,...,3.8,3.2,3.800000,4.166667,3.2,3.8,3.800000,3.273057,3.471470,3.360899


In [ ]:
# Test_result.info()
row_index=0
min_column =Test_result.iloc[row_index].idxmin()
print(min_column)

TypeError: reduction operation 'argmin' not allowed for this dtype

In [ ]:
#SMA10 Should be selected
# Therefore its prediction will be as follows
add_rolling(df_forecast, 10, 1, ['STORE_ID', 'PRODUCT_ID'], 'SALE_QTY', 'SALE_QTY', 'dayofyear')
newdataset=df_forecast['SALE_QTY'].tail(10)
newdataset.head()
newdataset=pd.DataFrame(newdataset)
newdataset = newdataset['SALE_QTY'].rolling(window=10, min_periods=1).mean()

# Shift the SMA values by 1 day
sma_10_shifted = sma_10.shift()

# Repeat the shifted SMA values for the next 10 days
newdataset = newdataset.reindex(range(len(df), len(df) + 10)).ffill()

# Add the shifted SMA values to the original series
df['sma_10_values'] = sma_10_repeated
# Test_result.loc[0,'Pred_Day1']=df_forecast.tail(10)

NameError: name 'sma_10' is not defined

In [ ]:
# Plot SMA 10

graph_df=pd.DataFrame()
graph_df['SALE_QTY']=test_data_forecast.groupby(['CALDAY'])['SALE_QTY'].sum()
graph_df['SMA10']=test_data_forecast.groupby(['CALDAY'])['SMA10'].sum()
graph_df.head()
# graph_df_sma10


NameError: name 'test_data_forecast' is not defined

In [ ]:
graph_df

,SALE_QTY,SMA10
CALDAY,,
2024-04-09,5669.0,3599.4
2024-04-10,5330.0,3709.4
2024-04-11,5660.0,3799.8
2024-04-12,5385.0,3885.8
2024-04-13,6583.0,4208.5
2024-04-14,6659.0,4754.7
2024-04-15,5228.0,4161.0
2024-04-16,5254.0,4141.3
2024-04-17,5478.0,4360.8


In [ ]:
MAPE_EVAL(graph_df, 'SMA10')


    MAPE   : 0.0
    MAPE % : 0.25 %
    


In [ ]:
# SMA 5 Days

df_forecast=add_rolling(df_forecast,5,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','SMA5','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]
# test_data_forecast.head(5)

NameError: name 'add_rolling' is not defined

In [ ]:
# Plot SMA 5


graph_df['SMA5']=test_data_forecast.groupby(['CALDAY'])['SMA5'].sum()
# graph_df_sma10


In [ ]:
# SMA 30 Days

df_forecast=add_rolling(df_forecast,30,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','SMA30','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]

In [ ]:
# Plot SMA 30


graph_df['SMA30']=test_data_forecast.groupby(['CALDAY'])['SMA30'].sum()
# graph_df_sma10


In [ ]:
# SMA 15 Days

df_forecast=add_rolling(df_forecast,15,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','SMA15','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]


NameError: name 'add_rolling' is not defined

In [ ]:
# Plot SMA 15

graph_df['SMA15']=test_data_forecast.groupby(['CALDAY'])['SMA15'].sum()
# graph_df_sma10


NameError: name 'test_data_forecast' is not defined

In [ ]:
def add_cummulative5(df, window=5, min_periods=1, indexes=['STORE_ID','PRODUCT_ID'],
                value='SALE_QTY', col='SMA10', time_col='dayofyear'):

    df.sort_values([time_col], inplace=True)
    tmp = df.groupby(indexes)[value].expanding(window).mean().shift()

    tmp.reset_index([i for i in range(len(indexes))], drop=True, inplace=True)
    tmp = tmp.to_frame()
    df.sort_index(inplace=True)
    tmp.sort_index(inplace=True)
    tmp.columns=[col]
    tmp = pd.concat([df, tmp], axis=1)

    tmp.sort_values([time_col], inplace=True)

    return tmp

In [ ]:
#Cummulative Mean Avg
def add_cummulative(df, window=10, min_periods=1, indexes=['STORE_ID','PRODUCT_ID'],
                value='SALE_QTY', col='SMA10', time_col='dayofyear'):

    df.sort_values([time_col], inplace=True)
    tmp = df.groupby(indexes)[value].expanding(window).mean().shift()

    tmp.reset_index([i for i in range(len(indexes))], drop=True, inplace=True)
    tmp = tmp.to_frame()
    df.sort_index(inplace=True)
    tmp.sort_index(inplace=True)
    tmp.columns=[col]
    tmp = pd.concat([df, tmp], axis=1)

    tmp.sort_values([time_col], inplace=True)

    return tmp

In [ ]:
def add_cummulative15(df, window=15, min_periods=1, indexes=['STORE_ID','PRODUCT_ID'],
                value='SALE_QTY', col='SMA10', time_col='dayofyear'):

    df.sort_values([time_col], inplace=True)
    tmp = df.groupby(indexes)[value].expanding(window).mean().shift()

    # tmp.reset_index([i for i in range(len(indexes))], drop=True, inplace=True)
    # tmp = tmp.to_frame()
    # df.sort_index(inplace=True)
    # tmp.sort_index(inplace=True)
    # tmp.columns=[col]
    # tmp = pd.concat([df, tmp], axis=1)

    # tmp.sort_values([time_col], inplace=True)

    return tmp

In [ ]:
# CMA 5 Days

df_forecast=add_cummulative(df_forecast,5,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','CMA5','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
df_forecast['CMA5'] = df_forecast['CMA5'].fillna(0)

test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]


In [ ]:
# Plot CMA 5


graph_df['CMA5']=test_data_forecast.groupby(['CALDAY'])['CMA5'].sum()
# graph_df_sma10


NameError: name 'graph_df' is not defined

In [ ]:
# CMA 10 Days

df_forecast=add_cummulative(df_forecast,5,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','CMA10','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
df_forecast['CMA10'] = df_forecast['CMA10'].fillna(0)

test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]


In [ ]:
# Plot CMA 10


graph_df['CMA10']=test_data_forecast.groupby(['CALDAY'])['CMA10'].sum()
# graph_df_sma10



NameError: name 'graph_df' is not defined

In [ ]:
# CMA 15 Days

df_forecast=add_cummulative(df_forecast,5,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','CMA15','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
df_forecast['CMA15'] = df_forecast['CMA15'].fillna(0)

test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]


In [ ]:
# Plot CMA 15


graph_df['CMA15']=test_data_forecast.groupby(['CALDAY'])['CMA15'].sum()
# graph_df_sma10



In [ ]:
# EWMA 5 Days

df_forecast=exponetial_weighted_ma(df_forecast,5,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA5','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
df_forecast['EWMA5'] = df_forecast['EWMA5'].fillna(0)

test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]


NameError: name 'exponetial_weighted_ma' is not defined

In [ ]:
# Plot EWMA 5

graph_df['EWMA5']=test_data_forecast.groupby(['CALDAY'])['EWMA5'].sum()



NameError: name 'test_data_forecast' is not defined

In [ ]:
# EWMA 10 Days

df_forecast=exponetial_weighted_ma(df_forecast,10,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA10','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
df_forecast['EWMA10'] = df_forecast['EWMA10'].fillna(0)

test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]


NameError: name 'exponetial_weighted_ma' is not defined

In [ ]:
# Plot EWMA 10
graph_df['EWMA10']=test_data_forecast.groupby(['CALDAY'])['EWMA10'].sum()



NameError: name 'test_data_forecast' is not defined

In [ ]:
# EWMA 30 Days
df_forecast=exponetial_weighted_ma(df_forecast,30,1,['STORE_ID','PRODUCT_ID'],'SALE_QTY','EWMA30','dayofyear')

# Remove NULL values
# df_max.dropna(inplace=True)
df_forecast['EWMA30'] = df_forecast['EWMA30'].fillna(0)

test_data_forecast=df_forecast[df_forecast['dayofyear']>=100]


NameError: name 'exponetial_weighted_ma' is not defined

In [ ]:
# Plot EWMA 10

graph_df['EWMA30']=test_data_forecast.groupby(['CALDAY'])['EWMA30'].sum()



NameError: name 'graph_df' is not defined

In [ ]:
test=test_data_forecast
test_graph=graph_df

In [ ]:
test_data_forecast.head(5)

,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY,product_store,year,month,day,dayofyear,dayofweek,...,SMA10,SMA5,SMA30,SMA15,CMA5,CMA10,CMA15,EWMA5,EWMA10,EWMA30
82152,2024-04-09,10625448,5338,1.0,10625448_5338,2024,4,9,100,1,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
82159,2024-04-09,10007927,2061,2.0,10007927_2061,2024,4,9,100,1,...,NaN,4.2,NaN,NaN,3.000000,3.000000,3.000000,3.629416,3.357393,3.128670
82154,2024-04-09,10195647,5570,5.0,10195647_5570,2024,4,9,100,1,...,NaN,9.8,NaN,NaN,9.800000,9.800000,9.800000,9.018061,9.428724,9.682705
82155,2024-04-09,10284315,5378,1.0,10284315_5378,2024,4,9,100,1,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
82156,2024-04-09,10197686,5547,1.0,10197686_5547,2024,4,9,100,1,...,NaN,1.4,NaN,NaN,1.166667,1.166667,1.166667,1.400470,1.287861,1.207934


In [ ]:
test_data_forecast.CALDAY.unique()

array(['2024-04-09', '2024-04-10', '2024-04-11', '2024-04-12',
       '2024-04-13', '2024-04-14', '2024-04-15', '2024-04-16',
       '2024-04-17', '2024-04-18', '2024-04-19', '2024-04-20',
       '2024-04-21', '2024-04-22', '2024-04-23', '2024-04-24',
       '2024-04-25', '2024-04-26', '2024-04-27', '2024-04-28',
       '2024-04-29', '2024-04-30', '2024-05-01', '2024-05-02',
       '2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06',
       '2024-05-07', '2024-05-08'], dtype=object)

In [ ]:
def add_optimized_column(df, error_duration=7, indexes=['STORE_ID','PRODUCT_ID'],
                value='SALE_QTY', time_col='CALDAY'):
    columns = ['SMA10', 'SMA5', 'SMA30', 'SMA15', 'CMA5', 'CMA10', 'CMA15', 'EWMA5', 'EWMA10', 'EWMA30']


    def min_value(row):
        diffs = {col: np.abs(row[col] - row['SALE_QTY']) for col in columns}
#         MAPE = {col: 100*(np.abs(row[col] - row['SALE_QTY'])/row['SALE_QTY']) for col in columns}
        min_col = min(diffs, key=diffs.get)
        return (round(row[min_col],0) )
    def min_method(row):
        diffs = {col: np.abs(row[col] - row['SALE_QTY']) for col in columns}
#         MAPE = {col: 100*(np.abs(row[col] - row['SALE_QTY'])/row['SALE_QTY']) for col in columns}
        min_col = min(diffs, key=diffs.get)
        return (min_col )
    def min_mape(row):
        diffs = {col: np.abs(row[col] - row['SALE_QTY']) for col in columns}
#         MAPE = {col: 100*(np.abs(row[col] - row['SALE_QTY'])/row['SALE_QTY']) for col in columns}
        min_col = min(diffs, key=diffs.get)
        return (MAPE.get(min_col))

    def get_macro_mape(l):
        losses = {}
#         print(l)
        for i in l:
            for j in columns:
                if j not in losses:
                    losses[j] = [i[j][0], i[j][1]]
                else:
                    losses[j][0] += i[j][0]
                    losses[j][0] += i[j][1]

        min_ = losses[columns[0]][0] / losses[columns[0]][1]
        strat = columns[0]

        for i in losses:
            p = losses[i][0] / losses[i][1]
            if p < min_:
                min_ = p
                strat = i
        return {'Methodology':strat,'MAPE':100*min_}

    def get_error(row):
        diffs = {col: (np.abs(row[col] - row['SALE_QTY']), row['SALE_QTY']) for col in columns}
        return diffs


    df['diffs'] = df.apply(get_error, axis=1)
#     df['Methodology'] = df.apply(min_method, axis=1)
#     df['MAPE'] = df.apply(min_mape, axis=1)

    if 'Methodology' in df:
        del df['Methodology']

    if 'Forecasted_Value' in df:
        del df['Forecasted_Value']

    if 'MAPE' in df:
        del df['MAPE']

    tmp = df.groupby(indexes)['diffs'].transform(lambda x: x.rolling(error_duration+1, 1)).to_frame().apply(
        lambda x: get_macro_mape(list(x['diffs'])), axis=1, result_type='expand')#.to_frame()

    df = pd.merge(df, tmp, left_index=True, right_index=True)
    df.drop(columns=['diffs'], inplace=True)


    df['Forecasted_Value'] = df.apply(lambda x: x[x['Methodology']], axis=1)

#     df['']


    return df

In [ ]:
test1=add_optimized_column(test_data_forecast, 9, ['STORE_ID','PRODUCT_ID'], 'SALE_QTY')
test1.head(5)

/var/folders/kf/drhxphyd4jzb33ltxtn284kr0000gp/T/ipykernel_15364/2769931050.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diffs'] = df.apply(get_error, axis=1)
/var/folders/kf/drhxphyd4jzb33ltxtn284kr0000gp/T/ipykernel_15364/2769931050.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  min_ = losses[columns[0]][0] / losses[columns[0]][1]
/var/folders/kf/drhxphyd4jzb33ltxtn284kr0000gp/T/ipykernel_15364/2769931050.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  p = losses[i][0] / losses[i][1]
/var/folders/kf/drhxphyd4jzb33ltxtn284kr0000gp/T/ipykernel_15364/2769931050.py:33: RuntimeWarning: invalid value encountered in double_scalars
  min_ = losses[columns[0]][0] / losses[columns[0]][1]
/var/folde

,CALDAY,PRODUCT_ID,STORE_ID,SALE_QTY,product_store,year,month,day,dayofyear,dayofweek,...,SMA15,CMA5,CMA10,CMA15,EWMA5,EWMA10,EWMA30,Methodology,MAPE,Forecasted_Value
82152,2024-04-09,10625448,5338,1.0,10625448_5338,2024,4,9,100,1,...,NaN,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,SMA10,NaN,NaN
82159,2024-04-09,10007927,2061,2.0,10007927_2061,2024,4,9,100,1,...,NaN,3.000000,3.000000,3.000000,3.629416,3.357393,3.128670,SMA10,NaN,NaN
82154,2024-04-09,10195647,5570,5.0,10195647_5570,2024,4,9,100,1,...,NaN,9.800000,9.800000,9.800000,9.018061,9.428724,9.682705,SMA10,NaN,NaN
82155,2024-04-09,10284315,5378,1.0,10284315_5378,2024,4,9,100,1,...,NaN,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,SMA10,NaN,NaN
82156,2024-04-09,10197686,5547,1.0,10197686_5547,2024,4,9,100,1,...,NaN,1.166667,1.166667,1.166667,1.400470,1.287861,1.207934,SMA10,NaN,NaN


In [ ]:
test1.Methodology.value_counts()

NameError: name 'test1' is not defined

In [ ]:
test_graph['Optimized_Value']=test1.groupby(['CALDAY'])['Forecasted_Value'].sum()
# df_fin=test_graph.tail(7)

NameError: name 'test1' is not defined

In [ ]:
test_graph=test_graph.dropna()
test_graph

NameError: name 'test_graph' is not defined

In [ ]:
print(MAPE_EVAL(test_graph,'Optimized_Value'))


    MAPE   : 0.06
    MAPE % : 6.48 %
    
None


In [ ]:
print(BIAS_EVAL(test_graph.tail(7),'Optimized_Value'))


    Bias % : -6.48 %
    
None


In [ ]:
df2=pd.read_csv('/content/result.csv')

In [ ]:
left_join_df = pd.merge(df1, df2, on='ID', how='left')
print("\nLeft Join DataFrame:")
print(left_join_df)

# Perform a right join
right_join_df = pd.merge(df1, df2, on='ID', how='right')
print("\nRight Join DataFrame:")
print(right_join_df)

# Perform an outer join
outer_join_df = pd.merge(df1, df2, on='ID', how='outer')
print("\nOuter Join DataFrame:")
print(outer_join_df)